
# Homework 1: Intro to Regular Expressions and Tokenization
## Total Points: 66 points
- **Overview**: This week we will start writing some code! We will be using Python Notebooks for most of this course, and this assignment examines your skills writing regular expressions and string processing in Python.

- **Relevant Textbook**: This homework assignment corresponds to the reading in _Speech and Language Processing_ Chapter 2: [Regular Expressions, Text Normalization, Edit Distance](https://web.stanford.edu/~jurafsky/slp3/2.pdf).

- **Grading**: We will use the auto-grading system called `PennGrader`. To complete the homework assignment, you should implement anything marked with `#TODO` and run the cell with `#PennGrader` note. **There will be no hidden tests in this assignment.** In other words, you will know your score once you finish all the `#TODO` and run all the `#PennGrader` tests!
## To get started, **make a copy** of this colab notebook into your google drive!

## Setup 1: PennGrader Setup - 4 points

In [1]:
## DO NOT CHANGE ANYTHING, JUST RUN
%%capture
!pip install huggingface_hub datasets --upgrade

In [2]:
## DO NOT CHANGE ANYTHING, JUST RUN
%%capture
!pip install penngrader-client

In [3]:
%%writefile notebook-config.yaml

grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

Writing notebook-config.yaml


In [4]:
!cat notebook-config.yaml


grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'


In [5]:
from penngrader.grader import *

## TODO: enter your Penn-ID
STUDENT_ID = 56803282 # YOUR PENN-ID GOES HERE AS AN INTEGER#

SECRET = STUDENT_ID
grader = PennGrader('notebook-config.yaml', 'CIS5300_OL_23Su_HW1', STUDENT_ID, SECRET)

PennGrader initialized with Student ID: 56803282

Make sure this correct or we will not be able to store your grade


In [6]:
# check if the PennGrader is set up correctly
# do not chance this cell, see if you get 4/4!
name_str = 'Jacky Choi' # TODO: enter your name
grader.grade(test_case_id = 'name_test', answer = name_str)

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## Setup 2: Packages
- **Run the following cells without changing anything!**

In [7]:
from tqdm import tqdm
import numpy as np
import copy
import requests
import re
from collections import defaultdict
from dill.source import getsource

In [8]:
import nltk
from nltk import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Section 1. Regular Expressions Basics [15 points]


- [Regular Expression Cheatsheet](https://cheatography.com/davechild/cheat-sheets/regular-expressions/)
- [Python Official RE Guide](https://docs.python.org/3/library/re.html)
- [W3School Re Guide](https://www.w3schools.com/python/python_regex.asp)
- [Regex Exercises](https://www.w3resource.com/python-exercises/re/)

![Benjamin Bannekat](https://www.pixelsham.com/wp-content/uploads/2020/05/REGEX.png)

In this section, we will explore basic usage of regular expressions in natural language processing, including three main functionalities:
- `search`: find the first matching pattern (if there is a match anywhere in the string)
- `findall`: find all of the matching patterns
- `sub`: replaces one or many matches with a string

In [9]:
### Helper function ###
### DO NOT CHANGE ###
from bs4 import BeautifulSoup
def get_text_from_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, features="html.parser")
    for script in soup(["script", "style"]):
        script.extract()
    text = soup.get_text()

    text = re.sub('\n+', '\n', text)
    return text
### DO NOT CHANGE ###

## 1.1 re.search

Your very first task is utilizing `re.search()` to find specific patterns from a text.

- **Problem 1.1:** search for the word "food"
    - **Overview**: Implement the function `find_food()` that determines if the word "food" appears in the string.
    - **Input**: a string
    - **Output**: `True` or `False`
    - **Requirements**:
        - Functionality: The function returns True if the word "food" is found in the input string, and False otherwise. It also accepts **potential typos**, e.g. "fooood".
        - To be specific, valid "food" entry should have 1) one "f" at the beginning 2) two or more "o"s 3) one "d" at the end
    - **Examples**:
        - `find_food("foooood")`: `True`
        - `find_food("foodddd")`: `False`
        - `find_food("fod")`: `False`
        - `find_food("food!!!")`: `True`
        - `find_food("I like fooood!!")`: `True`
        - `find_food("foodie")`: `False`

In [10]:
def find_food(text):
    """
    Search for the word "food" in the text. More than two "o"s is also accepted.
    Use re.search()

    @param text: the string to be searched
    @return: True if "food" is found, False if not
    """
    # TODO
    pattern = 'fo{2,}d(?![a-z])'
    return bool(re.search(pattern, text))

In [11]:
print(find_food("foooood"))
print(find_food("foodddd"))
print(find_food("fod"))
print(find_food("food!!!"))
print(find_food("I like fooood!!"))
print(find_food("foodie"))

True
False
False
True
True
False


In [12]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q1_food_regex_function', answer = getsource(find_food))

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 1.2 re.findall
The second exercise is using `re.findall()` to find all non-overlapping matches of pattern in string.

As an example, we will first take a look at this [website](https://www.presidentsusa.net/birth.html), which stores all the birthdays of U.S presidents. We will try to parse all the birthdays from this website.

In [13]:
### Read text data ###
### DO NOT CHANGE ###
web_text = get_text_from_url('https://www.presidentsusa.net/birth.html')
print(web_text[:500]) # let's print the first 500 char
### DO NOT CHANGE ###

ï»¿
List of Presidential Birthplaces, Birthdate, and Death information
 
 
List of U.S. Presidents Birthplaces and Location of Death
President
Birth Date
Birth Place
Death Date
Location of Death
George Washington
Feb 22, 1732
Westmoreland
                                    Co., Va.
Dec 14, 1799
Mount Vernon, Va.
John Adams
Oct 30, 1735
Quincy, Mass.
July 4, 1826
Quincy, Mass.
Thomas Jefferson
Apr 13, 1743
 Albemarle Co., Va.
July 4, 1826
 Albemarle Co., Va.
James Madison
Mar 16, 1751
Port Conw


In [14]:
# find all the birthdays in the website
date_pattern = '[A-Za-z]{3,4} \d{1,2}, \d{4}' # can you understand what it is trying to do?
all_dates = re.findall(date_pattern, web_text)
len(all_dates) # how many birthdays are there?

85

In [15]:
all_dates[:5] # check out some of the matches!

['Feb 22, 1732',
 'Dec 14, 1799',
 'Oct 30, 1735',
 'July 4, 1826',
 'Apr 13, 1743']

Now it's your turn. Please use `re.findall()` and try to to parse email addresses.

- **Problem 1.2:** search for emails
    - **Overview**: Implement the function `find_email()` to find **all the emails** from the input text. We will look for emails of the faculties at [Penn Engineering Online](https://online.seas.upenn.edu/about/faculty-directory/)
    - **Input**: a string
    - **Output**: a list of strings
    - **Requirements**:
        - Return a list of valid emails found in the input string
        - You may assume all the emails end with `.edu` or `.com`
        - You may use no more than 2 regex patterns
    - **Caveat**: When using `findall()`, be careful of using parentheses. Anything you have in parentheses () will be a capture group, which may have different behavior than you expect. Check [the documentation](https://docs.python.org/3/howto/regex.html) for detailed explanation.

In [16]:
### Read text data ###
### DO NOT CHANGE ###
url = 'https://online.seas.upenn.edu/about/faculty-directory/'
penn_online_text = get_text_from_url(url)
print(penn_online_text[1620:2600]) # check what it looks like
### DO NOT CHANGE ###

Directory
Home
            About Us          
Faculty Directory
              About Us            
Faculty Leadership 
Boon Thau Loo
RCA Professor, Department of Computer and Information Science
Senior Associate Dean for Education and Global Initiatives — School of Engineering and Applied Science
Director, Distributed Systems Laboratory
Email: boonloo@cis.upenn.edu 
Tom Farmer
Program Director, Online Master of Computer and Information Technology
Senior Lecturer, Department of Electrical and Systems Engineering & Department of Computer and Information Science
Email: tfarmer@seas.upenn.edu 
James C. Gee
Program Director, Online Master of Science in Engineering in Data Science
Professor, Department of Radiology (Perelman School of Medicine) and Department of Computer and Information Science (School of Engineering and Applied Science)
Director, Penn Image Computing and Science Laboratory | Co-Director, Translational Biomedical Imaging Center | Director, Interfaces Prog


In [17]:
def find_email(text):
    """
    Find all the valid email addresses from the text, which ends with .edu or .com
    Use re.findall() with no more than 2 patterns

    @param text: the string to be searched
    @return: a list of valid emails
    """
    # TODO
    email_lst = '[a-z]{1,10}@?(?:cis.|seas.|\@)upenn.edu'
    return re.findall(email_lst,text)

In [18]:
find_email(penn_online_text)

['boonloo@cis.upenn.edu',
 'tfarmer@seas.upenn.edu',
 'gee@upenn.edu',
 'ccb@upenn.edu',
 'ahae@cis.upenn.edu',
 'kimbong@seas.upenn.edu',
 'ewaingar@cis.upenn.edu',
 'hassani@seas.upenn.edu',
 'sharry@seas.upenn.edu',
 'jacobkahn@seas.upenn.edu',
 'jean@cis.upenn.edu',
 'jshi@cis.upenn.edu',
 'mhnaik@cis.upenn.edu',
 'nanzheng@seas.upenn.edu',
 'pratikac@seas.upenn.edu',
 'kannan@cis.upenn.edu',
 'sanjeev@cis.upenn.edu',
 'venkatesh@seas.upenn.edu',
 'sharathg@seas.upenn.edu',
 'mza@seas.upenn.edu',
 'susan@cis.upenn.edu',
 'val@cis.upenn.edu',
 'preciado@seas.upenn.edu',
 'zives@cis.upenn.edu']

In [19]:
emails_found = find_email(penn_online_text)
assert 'ccb@upenn.edu' in emails_found, "CCB's email not found" # this should return nothing if CCB's email is found

In [20]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q12_email_regex_function', answer = emails_found)

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 1.3 re.sub


In this problem, we will work with the Wikipedia webpage containing a [List of people from New Jersey](https://simple.wikipedia.org/wiki/List%20of%20people%20from%20New%20Jersey).

In this website, people who are still living are originally labeled with their birth year. Here, your task is to use the `re.sub()` function to label them as "still active nowadays". For example, "Gerard Way (born 1977)" should be updated to "Gerard Way (still active nowadays)".

Please note that we don't want to change the format for people who are no longer living, such as "Sarah Vaughan (1924–1990)"


In [21]:
### Read text data ###
### DO NOT CHANGE  ###
q13_text = get_text_from_url('https://simple.wikipedia.org/wiki/List_of_people_from_New_Jersey')
print(q13_text[1913:2739]) # check what it looks like
### DO NOT CHANGE  ###

ve Forbes
Todd Frazier
Isa Abdul-Quddus (born 1988), safety for the Detroit Lions (Union)
Joseph Alexander Adams (1803–1880), engraver (born in New Germantown)[1]
Mike Adams (born 1981), safety for the Indianapolis Colts (Paterson)
Timothy Adams (born 1967), actor, Sunset Beach, Ocean Ave. (Belleville)
Charles Addams (1912-1988), cartoonist; creator of The Addams Family (Westfield)
Jordan Alan (born 1967), filmmaker (Bayonne)
Mitch Albom (born 1958), writer, broadcaster, and musician (Passaic)
Edwin "Buzz" Aldrin (born 1930) NASA, astronaut, second man to walk on the moon (born in Glen Ridge, grew up in Montclair)
Jason Alexander (born 1959), actor, George Costanza on Seinfeld (Newark, raised in Livingston)
Samuel Alito (born 1950), U.S. Supreme Court justice (Trenton, raised in Hamilton)
Malik Allen (born 1978), N


- **Problem 1.3**: substitute date format for all active people
    - **Overview**: Implement the function `change_date_format()` to replace all "(born in xxxx)" with "(still active nowadays)" in the input text.
    - **Input**: text that contains information of notable people from [New Jersey](https://simple.wikipedia.org/wiki/List%20of%20people%20from%20New%20Jersey) (`q13_text`)
    - **Output**: the modified document as a string
    - **Example**:  

```
"...
Isa Abdul-Quddus (born 1988), safety for the Detroit Lions (Union)
Joseph Alexander Adams (1803–1880), engraver (born in New Germantown)[1]
Mike Adams (born 1981), safety for the Indianapolis Colts (Paterson)
..."
```


->


```
"...
Isa Abdul-Quddus (still active nowadays), safety for the Detroit Lions (Union)
Joseph Alexander Adams (1803–1880), engraver (born in New Germantown)[1]
Mike Adams (still active nowadays), safety for the Indianapolis Colts (Paterson)
..."
```



In [22]:
def change_date_format(text):
    """
    Replace all the "(born in xxxx)" with "(still active nowadays)" in the text.
    Use re.sub()

    @param text: the string to be searched
    @return: the modified document
    """
    # TODO
    modified_text = re.sub('\(born \d{4}\)', '(still active nowadays)', text)
    return modified_text

In [23]:
q13_text_sub = change_date_format(q13_text)
assert "born 19" not in q13_text_sub[1913:2785], "not correctly modified"
assert "still active nowadays" in q13_text_sub[1913:2785], "not correctly modified"

In [24]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q13_sub_regex_function', answer = q13_text_sub)

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


# Section 2. Tokenization [47 points]

## 2.1 Naive Way

### 2.1.1 Whitespace Tokenizer - 2 points
Word tokenization is the process of splitting a large sample of text into words. This is a requirement in natural language processing tasks where each word needs to be captured and subjected to further analysis like classifying and counting them for a particular sentiment etc.

Whitespace Tokenizer is the most basic tokenizer which splits strings on ICU-defined whitespace characters (eg. space, tab, new line).
This is often good for quickly building out prototype models.

- **Problem 2.1.1** Whitespace Tokenizer
  - **Overview**: Implement the function `whitespaceTokenizer()` to tokenize the sentence by splitting with whitespace characters
  - **Input**: a string
  - **Output**: a list of strings
  - **Requirements**:
      - Split the input text by whitespaces and return a list of tokens
      - The function should also be able to recognize other forms of whitespaces, such as `\n`
      - Please only use Python's built-in functions. NLTK and other packages should not be used here
  - **Example**: `"This is a test"` -> `["This", "is", "a", "test"]`

In [25]:
def whitespaceTokenizer(text):
    """
    Tokenize the sentence by splitting with whitespace characters
    Use built-in functions only

    @param text: the string to be tokenized
    @return: a list of strings
    """
    # TODO
    tokens = text.split()
    return tokens

In [26]:
whitespaceTokenizer("This is a\n test")

['This', 'is', 'a', 'test']

In [27]:
text = "This is a test"
assert whitespaceTokenizer(text) == ["This", "is", "a", "test"]

In [28]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q211_whitespace_tokenizer_function', answer = getsource(whitespaceTokenizer))

Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### 2.1.2 Update Whitespace Tokenizer - 2 points

We see that seperating words with spaces works sometimes, but what if there are punctuations? With the most basic implementation in 2.1.1, `"This, is a test!"` will become `["This,", "is", "a", "test!"]` after tokenization. This is not what we usually want. Punctuation is often treated as a special token. For instance, periods and exclamation points mark the end of a sentence. Therefore, naturally we want them to be separated with words during tokenization.

Please implement `whitespaceTokenizerUpdate()` as an updated version that separates all the punctuations when splitting words with whitespaces.

- **Problem 2.1.2** Update Whitespace Tokenizer
  - **Overview**: Implement the function `whitespaceTokenizerUpdate()` to tokenize the sentence by splitting the whitespace characters and separating punctuation at the same time
  - **Input**: a string
  - **Output**: a list of strings
  - **Requirements**:
      - Split the input text by whitespaces and return a list of tokens
      - The function should also be able to recognize other forms of whitespaces, such as `\n`, and separate punctuations from words
      - Please only use Python's built-in functions. NLTK and other packages should not be used here
      - You may assume there are only common English punctuations
  - **Example**: `"This 212, is a test!"` -> `["This", "212", "," "is", "a", "test", "!"]`

In [29]:
def whitespaceTokenizerUpdate(text):
    """
    Tokenize the sentence by splitting with whitespace characters
    Punctuations should be separated

    @param text: the string to be tokenized
    @return: a list of strings
    """
    # TODO
    tokens = re.findall('\w+|[^\w\s]', text)
    return tokens

In [30]:
whitespaceTokenizerUpdate("This 212, is a test!")

['This', '212', ',', 'is', 'a', 'test', '!']

In [31]:
text = "This 212, is a test!"
assert whitespaceTokenizerUpdate(text) == ["This", "212", ",", "is", "a", "test", "!"]

In [32]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q212_whitespace_tokenizer_update', answer = getsource(whitespaceTokenizerUpdate))

Correct! You earned 2/2 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### 2.1.3 Tokenize with NLTK Library - 4 points
You must have noticed that tokenization can be very tricky depending on your needs. With the updated whitespace tokenizer, you could probably get desirable results in 95% of the tasks with someting super simple, and 99% with an extra 20-60 minutes of updating. It may require a lot more to get to 99.9%.

Luckily, there are many useful resources from the NLTK library that can help with this task.


- **Problem 2.1.3** NLTK's word tokenizer.
  - **Overview**: Implement the function `nltk_tokenize_content()` to tokenize sentences
  - **Input**: a string
  - **Output**: a list of strings
  - **Requirements**:
      - Convert the input text to lower case
      - Utilize NLTK's `work_tokenize()` function
      - Remove all the non-words and stop words (hint: non-words: words are not purely alphabetic. In other words, a 'word' should be purely alphabetic.)
  - **Example**: `"Good muffins cost $3.88\nin New (York).  Please (buy) me\ntwo of them.\n(Thanks)."` -> `['good', 'muffins', 'cost', 'new', 'york', 'please', 'buy', 'two', 'thanks']`

In [33]:
### Download Resources ###
### DO NOT CHANGE ###
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')
stopwords = set(stopwords.words('english'))
### DO NOT CHANGE ###

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [34]:
def nltk_tokenize_content(text):
    """
    Tokenize the input text with nltk's word tokenizer
    Convert the text to lowercase and remove all non-words and stopwords
    see https://www.nltk.org/howto/tokenize.html

    @param text: a string for tokenization
    @return: a list of tokens
    """
    # TODO
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in stopwords]
    return tokens

In [35]:
text = "Good muffins cost $3.88\nin New (York).  Please (buy) me\ntwo of them.\n(Thanks)."
assert nltk_tokenize_content(text) == ['good', 'muffins', 'cost', 'new', 'york', 'please', 'buy', 'two', 'thanks']

In [36]:
### Test cases ###
### DO NOT CHANGE ###
test_sents = [
    "The 19-year-old singer was caught on camera being escorted out of the store by security guards.",
    "“A program is never less than 90% complete, and never more than 95% complete.”— Terry Baker's quote'",
    "To train such a large model, OpenAI crawled 40GB worth of text from the web (roughly 20,000,000,000 words).",
    "text = 'God is Great! I won a lottery.'; 'id': 22189,"
]
test_results = [nltk_tokenize_content(sent) for sent in test_sents]
### DO NOT CHANGE ###

# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q213_nltk_tokenizer_function', answer = test_results)

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


As a side note, one commonly used tokenization standard is known as the Penn Treebank tokenization standard, which uses regular expressions to tokenize text.

This standard has the following features:

- split standard contractions, e.g. doesn't -> does & n't; they'll -> they & 'll
- keep hyphenated words together
- treat most punctuation characters as separate tokens
- split off commas and single quotes, when followed by whitespace
- separate periods that appear at the end of a line

In practice, since tokenization needs to be run before any other language processing, it needs to be very fast. The standard method for tokenization is therefore to use deterministic algorithms based on regular expressions compiled into very efficient finite state automata.

With regular expressions, it is very convenient to customize the tokenization algorithm with the NLTK library. For example, instead of removing all stop words and non-word tokens, we may want to keep a selective set of tokens. e.g. percentages and complex words:


In [37]:
### EXAMPLE ###
### DO NOT CHANGE ###
text = 'That U.S.A. poster-print costs $12.40...'
pattern = r'''(?x)          # set flag to allow verbose regexps
        (?:[A-Z]\.)+        # abbreviations, e.g. U.S.A.
      | \w+(?:-\w+)*        # words with optional internal hyphens
      | \$?\d+(?:\.\d+)?%?  # currency and percentages, e.g. $12.40, 82%
      | \.\.\.              # ellipsis
      | [][.,;"'?():_`-]    # these are separate tokens; includes ], [
    '''
print(nltk.regexp_tokenize(text, pattern))
### DO NOT CHANGE ###

['That', 'U.S.A.', 'poster-print', 'costs', '$12.40', '...']


## 2.2 Tokenize with Stanza/allennlp Library - 5 points
[Stanza](https://stanfordnlp.github.io/stanza/tokenize.html) is another useful tool for word tokenization. Upon feeding it raw text, Stanza will:
- Tokenize it and groups tokens into sentences as the first step of processing
- Unlike other existing toolkits, Stanza combines tokenization and sentence segmentation from raw text into a single module.
(This is done to predict the position of words in a sentence, as use of words are context-sensitive in some languages.)


In [38]:
### Import stanza ###
### DO NOT CHANGE ###
%%capture
!pip install stanza==1.5.0
import stanza

nlp = stanza.Pipeline(lang='en', processors='tokenize')
### DO NOT CHANGE ###

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Done loading processors!


- **Problem 2.2**: Stanza tokenizer

  - **Overview**: Implement the function `stanza_tokenizer()` to tokenize a sentence using the stanza tokenizer.
  - **Input**: a string
  - **Output**: a list of strings
  - **Requirements**:
      - Utilize Stanza's pipeline (the variable `nlp` in the above cell)
      - Check the documentation [here](https://stanfordnlp.github.io/stanza/tokenize.html)
  - **Example**: `"Good muffins cost $3.88\nin New (York).  Please (buy) me\ntwo of them.\n(Thanks)."` -> `['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', '(', 'York', ')', '.', 'Please', '(', 'buy', ')', 'me', 'two', 'of', 'them', '.', '(', 'Thanks', ')', '.']`

In [39]:
def stanza_tokenizer(text):
    """
    Tokenize the input text with stanza's word tokenizer
    see https://stanfordnlp.github.io/stanza/tokenize.html

    @param text: the string for tokenization
    @return: a list of tokens
    """
    # TODO
    tokens = []
    doc = nlp(text)
    for sent in doc.sentences:
        for token in sent.tokens:
            tokens.append(token.text)
    return tokens

In [40]:
text = "Good muffins cost $3.88\nin New (York).  Please (buy) me\ntwo of them.\n(Thanks)."
assert stanza_tokenizer(text) == ['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', '(', 'York', ')', '.', 'Please', '(', 'buy', ')', 'me', 'two', 'of', 'them', '.', '(', 'Thanks', ')', '.']

In [41]:
### DO NOT CHANGE ###
test_sents = [
    'Antidisestablishmentarianism',
    'Conversationalization',
    "Most have seen this as Trump partisanizing the issue, much as he has with masks.",
    "“A program is never less than 90% complete, and never more than 95% complete.”— Terry Baker's quote'",
    "This is a test sentence for stanza. This is another sentence."
]
test_results = [stanza_tokenizer(sent) for sent in test_sents]
### DO NOT CHANGE ###

# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q22_stanza_tokenizer_function', answer = test_results)

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


## 2.3 Subword tokenizers (Byte-Pair Encoding) - 34 points

Whole word tokenization algorithms aren't perfect, because some languages have many word-forms and words modification for each word, such as prefixes and suffixes. We want to save morphology information in the text, but to save every possible word-form isn't memory-efficient or easy to train. Even then, we might still run into unknown words during inference.

Alternatively, subword can be used with a smaller vocabulary, and allow the model to have some information about novel words from the subwords that create them. We can create a tokenziation mechanism that will tokenize every word by subword morphology. The unsupervised algorithm that does so called is Byte Pair Encoding. Here is how it works:

![](https://lena-voita.github.io/resources/lectures/seq2seq/bpe/build_merge_table.gif)

1. Split text into individual characters
2. Begin BPE with a vocabulary that is just the set of all individual characters.
3. Choose the two symbols from the vocabulary that are most frequently adjacent in the corpus
4. Merge the most popular pair and update the corpus with the new merge pair
5. Continue to step 3 until we reach given vocabulary size.

It's easy algorithm, and we have several implementations:
- SentencePiece
- fastBPE
- Tokenizers by Huggingface🤗
- YouTokenToMe

We will now implement our BPE tokenization from scratch:

### 2.3.1 Wikipedia Simple Dataset Word Frequency - 5 points

We'll use this [Wikipedia Simple Dataset](https://huggingface.co/datasets/dangne/processed-wikipedia-20220301.simple) as our training data for BPE tokenization. To begin with, we need to compute word frequencies which will guide merging during tokenization.

- **Problem 2.3.1:** Compute word frequencies
    - **Overview**: Implement the function `compute_word_freq()` to compute the frequencies of each word in the Wikipedia Simple dataset.
    - **Input**: a list of strings (sentences), an int as threshold
    - **Output**: a dictionary with words as keys and counts as values
    - **Requirements**:
        - Iterate over the corpus
        - Tokenize each sentence with your `nltk_tokenize_content()` function in 2.1.3 to get a list of words
        - Count the frequency of each word
        - **Only keep a word if its frequency is equal or bigger than the threshold (50 by default)**
    - **Example**:
    

```
>>> lst = ['Good muffins cost $3.88\nin New (York).  Please (buy) me\ntwo of them.\n(Thanks).', \
'Amidst the vibrant streets of New York, a cozy bakery entices passersby with the scent of freshly baked blueberry muffins.']
>>> dic = compute_word_freq(lst, threshold = 2)
>>> print(dic)
{'muffins': 2, 'new': 2, 'york': 2}
```


In [42]:
### Load data ###
### DO NOT CHANGE ###
%%capture
!pip install datasets
from datasets import load_dataset
# see https://huggingface.co/docs/datasets/v1.8.0/loading_datasets.html#from-local-files
data = load_dataset("dangne/processed-wikipedia-20220301.simple")
### DO NOT CHANGE ###

In [43]:
def compute_word_freq(corpus, threshold = 50):
    """
    Compute the frequencies of each word in the corpus after tokenization
    Keep only words with frequencies >= threshold

    @param1 corpus: a list of strings
    @param2 threshold: an int indicating minimum frequency accepted
    @return: a dictionary with words as keys and frequencies as values
    """
    # TODO
    word_freqs = {}
    for sent in corpus: #iterate each sentnce
        tokens = nltk_tokenize_content(sent) #split sentence into puncs and words
        for token in tokens: #counts each word or places a new word dict in if thers none
            if token in word_freqs:
                word_freqs[token] += 1
            else:
                word_freqs[token] = 1
    word_freqs = {k: v for k, v in word_freqs.items() if v >= threshold} #check for threshold
    return word_freqs

In [44]:
compute_word_freq(['Good muffins cost $3.88\nin New (York).  Please (buy) me\ntwo of them.\n(Thanks).', \
'Amidst the vibrant streets of New York, a cozy bakery entices passersby with the scent of freshly baked blueberry muffins.'], 2)

{'muffins': 2, 'new': 2, 'york': 2}

In [45]:
lst = ['Good muffins cost $3.88\nin New (York).  Please (buy) me\ntwo of them.\n(Thanks).', \
'Amidst the vibrant streets of New York, a cozy bakery entices passersby with the scent of freshly baked blueberry muffins.']
dic = compute_word_freq(lst, threshold = 2)
assert dic == {'muffins': 2, 'new': 2, 'york': 2}

In [46]:
### Compute word_freqs ###
### DO NOT CHANGE ###
corpus = [dp['text'] for dp in data['train']]
word_freqs = compute_word_freq(corpus) # this line might take up to 10 minutes to finish, might be worth it to test it on a small corpus first (e.g. a few sentences)
### DO NOT CHANGE ###

In [47]:
assert len(word_freqs) == 24900

In [48]:
### DO NOT CHANGE ###
test_corpus = [
    "The cat and dog playfully chased each other around the yard, showing their playful nature.",
    "While the cat prefers to nap, the dog's favorite pastime is to play fetch in the park.",
    "At the pet-friendly cafe, you can watch as the cat and dog interact and play with their toys.",
    "The children laughed joyfully as they watched their pet cat and dog play together in the living room.",
    "In the early morning, you can often see the cat and dog playfully exploring the garden side by side."
]
test_word_freqs = compute_word_freq(test_corpus, 3)
### DO NOT CHANGE ###

# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q231_word_freq_function', answer = (test_word_freqs, word_freqs))

Correct! You earned 5/5 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### 2.3.2 Construct an Alphabet for the Corpus - 4 points

Now we have our corpus with all the frequencies. The next step is to build the vocabulary. As a very simple example, let's say our corpus uses these two words: `['april', 'proud']`, the base vocabulary would be `['a', 'd', 'i', 'l', 'o', 'p', 'r', 'u']`. For real-world cases, that base vocabulary will contain all the ASCII characters, at the very least, and probably some Unicode characters as well.

Why do we need the vocabulary? Recall that the most frequent pairs will be added to the base vocabulary during training. The BPE algorithm will keep running until we reach a certain vocabulary size.

For the convenience of training, here we will also split each word separately into individual characters. This will be used to compute the most frequent pairs.

- **Problem 2.3.2-1**: Build vocabulary
    - **Overview**: Implement The function `compute_vocab()` to build the base vocabulary for the corpus
    - **Input**: a dictionary with words as keys and counts as values (`word_freqs`)
    - **Output**: a list of characters
    - **Requirements**:
        - Sort the base vocabulary in lexicographic order.
        - Please don't modify `word_freqs` in-place.
    - **Example**: `{'april': 2, 'proud': 3}` -> `['a', 'd', 'i', 'l', 'o', 'p', 'r', 'u']`
- **Problem 2.3.2-2**: Split corpus
    - **Overview**: - Implement the function `corpus_splitter()` to split each word into individual characters in `word_freqs`
    - **Input**: a dictionary with words as keys and counts as values (`word_freqs`)
    - **Output**: a dictionary with words as keys and **a list of characters** as values
    - **Requirements**:
        - Please don't modify `word_freqs` in-place. Return a new dictionary
    - **Example**: `{'april': 2, 'proud': 3}` -> `{'april': ['a','p', 'r', 'i', 'l'], 'proud': ['p', 'r', 'o', 'u', 'd']}`


In [49]:
def compute_vocab(word_freqs):
    """
    Compute the base vocabulary from all words

    @param word_freqs: the dictionary we get in 2.3.1
    @return: a list of all unique characters
    """
    # TODO
    base_vocab = []
    for word in word_freqs:
        for c in word:
            if c not in base_vocab:
                base_vocab.append(c)
    base_vocab.sort()
    return base_vocab

In [50]:
base_vocab = compute_vocab(word_freqs)
assert len(base_vocab) == 74

In [53]:
def corpus_splitter(word_freqs):
    """
    Split each word into individual characters (i.e. list of characters)
    e.g. {april: [a, p, r, i, l]}

    @param word_freqs: the dictionary we get in 2.3.1
    @return: a dictionary in which keys are the words in word_freqs,
      values are lists of characters after splitting
    """
    # TODO
    splits = {}
    for word in word_freqs:
        splits[word] = list(word)


    return splits

In [54]:
splits = corpus_splitter(word_freqs)
assert splits['april'] == ['a', 'p', 'r', 'i', 'l']

In [55]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q232_alphabet_size_and_spliter', answer = (base_vocab, splits))

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### 2.3.3 Computer Token Pair Frequency - 4 points

Now that we have the splitted corpus, we can compute the frequencies for each pair. Let's assume the words have the following frequencies: `{'april': 2, 'proud': 3}`. Then we look at frequencies of all pairs such as `("a", "p")` and `("p", "r")`. The pair `("a", "p")` only appears in the word `'april'` so the frequency is also 2. The pair `("p", "r")` is present in both words, so 5 times total in the corpus.

It's very important to know that the the values in `splits` can include tokens with more than 1 character. For instance, after several rounds of training, the tokens of the word `'april'` could be `'april': ['ap', 'r', 'i', 'l']`. We will dig into the merge process in 2.3.4.

- **Problem 2.3.3**: compute pair frequency
    - **Overview**: Implement the function `compute_pair_freqs()` to compute the frequencies of **pairs of characters** as defined in the current vocabulary in the corpus.
    - **Input**: `word_freqs` from 2.3.1, `splits` from 2.3.2
    - **Output**: a dictionary with pairs (tuples) as keys and frequencies as values
    - **Requirements**:
        - Retrieve the pairs from `splits` and find their frequencies from `word_freqs`
        - Please don't modify `word_freqs` and `splits`in-place
    - **Example1**: `word_freqs = {'april': 2, 'proud': 3}; splits = {'april': ['a','p', 'r', 'i', 'l'], 'proud': ['p', 'r', 'o', 'u', 'd']}` -> `pair_freqs = {('a', 'p'): 2, ('p', 'r'): 5, ('r', 'i'): 2, ('i', 'l'): 2, ('r', 'o'): 3, ('o', 'u'): 3, ('u', 'd'): 3}`
    - **Example2**: `word_freqs = {'april': 2, 'apricot': 3}; splits = {'april': ['ap', 'r', 'i', 'l'], 'apricot': ['ap', 'r', 'i', 'c', 'o', 't']}` -> `pair_freqs = {('ap', 'r'): 5,('r', 'i'): 5, ('i', 'l'): 2, ('i', 'c'): 3, ('c', 'o'): 3, ('o', 't'): 3}`

In [56]:
def compute_pair_freqs(word_freqs, splits):
    """
    Compute the frequency of pairs of tokens

    @param1 word_freqs: the dictionary we get in 2.3.1
    @param2 splits: the dictionary we get in 2.3.2
    @return: a dictionary in which keys are pairs of tokens,
      values are frequencies
    """
    # TODO
    pair_freqs = {}
    for word in word_freqs:
        for i in range(len(splits[word]) - 1):
            pair = (splits[word][i], splits[word][i + 1])
            if pair not in pair_freqs:
                pair_freqs[pair] = word_freqs[word]
                continue
            pair_freqs[pair] += word_freqs[word]
    return pair_freqs

In [57]:
pair_freqs = compute_pair_freqs(word_freqs, splits)

In [58]:
# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q233_pair_freqs', answer = pair_freqs)

Correct! You earned 4/4 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### 2.3.4 Merge Token Pairs in Corpus - 6 points

After getting `pair_freqs`, we're able to find the most frequent pairs from it. As is mentioned previously, we will update `splits` by merge these pairs. Let's use the same example again. Suppose the `pair_freqs` is `{('a', 'p'): 2, ('p', 'r'): 5, ('r', 'i'): 2, ('i', 'l'): 2, ('r', 'o'): 3, ('o', 'u'): 3, ('u', 'd'): 3}`, we will find the most frequent pair is `('p', 'r')`. Then, the updated `splits` will be: `{'april': ['a','pr', 'i', 'l'], 'proud': ['pr', 'o', 'u', 'd']}`

- **Problem 2.3.4**: Merge pair
    - **Overview**: Implement the function `merge_pair` to merge a specified pair of tokens, and update the entire splits with the merge.
    - **Input**: the first and second tokens to be merged; a dictionary in the same format as `splits`
    - **Output**: the updated splits
    - **Hint**:
        - You can assume `(a, b)` is the most frequent pair. There's no need to find the max frequencies.
    - **Example**: `a = 'p', b = 'r', splits = {'april': ['a','p', 'r', 'i', 'l'], 'proud': ['p', 'r', 'o', 'u', 'd']}` -> `updated_splits = {'april': ['a','pr', 'i', 'l'], 'proud': ['pr', 'o', 'u', 'd']}`

In [59]:
def merge_pair(a, b, splits):
    """
    Merge the a and b into ab and return the updated splits dict
    This function will be used repeatedly

    @param1 a: the first token to be merged
    @param2 b: the second token to be merged
    @param3 splits: a dictionary in which keys are pairs of tokens,
      values are frequencies
    @return: the updated splits
    """
    # TODO
    updated_splits = {}
    ab = a + b

    for key, tokens in splits.items():
        new_tokens = []
        i = 0
        while i < len(tokens):
            if i < len(tokens) - 1 and tokens[i] == a and tokens[i + 1] == b:
                new_tokens.append(ab)
                i += 2
            else:
                new_tokens.append(tokens[i])
                i += 1
        updated_splits[key] = new_tokens
    return updated_splits

In [60]:
test_splits = {
    'april': ['a', 'p', 'r', 'i', 'l'],
    'apartment': ['a', 'p', 'a', 'r', 't', 'm', 'e', 'n', 't']
}
test_merge_result = merge_pair('a', 'p', test_splits)

# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q234_merge_pair_function', answer = test_merge_result)

Correct! You earned 6/6 points. You are a star!

Your submission has been successfully recorded in the gradebook.


### 2.3.5 Putting it all together - 15 points

With all the building blocks above, we're ready to train. Let's review the procedure and put everything together.

- **Problem 2.3.5-1**:
    - **Overview**: Implement the training loop for BPE, iteratively finding the most frequent pair, update the corpus by merging all adjacent tokens of that pair, and repeat.
    - **Input**: The base vocabulary: `base_vocab`; corpus: `word_freqs`; the splitted corpus: `splits`
    - **Output**: `merge_records`, a dictionary with pairs as keys and merged pairs as values. e.g., `{('to', 'y'): 'toy'}`
    - **Requirements**:
        -   For each iteration of training:
          -  Use `compute_pair_freqs()` to get the frequencies and find the most frequent pair
          -  Use `merge_pair()` to merge that pair in `splits`
          -  Save the learned merge rules in `merges_records`
          -  Update the vocabulary
          -  Repeat until the size of the vocabulary reach a certain number
        - **Use 5000 as the vocab size to pass the autograder, but before that, try using smaller vocab size to test your code.**



In [61]:
###### Part 1: Training ######
%%time
vocab_size = 5000 # test with smaller vocab size first, 5000 vocab would take 25-30 mins
vocab = base_vocab # initialize the base vocabulary
merges_records = {} # used to save the learned merge rules, e.g.: {('to', 'y'): 'toy'}
splits_copy = copy.deepcopy(splits) # work on a copy so that you can trace back

while len(vocab) < vocab_size: # repeat until vocab size reaches 5000
    # TODO: compute pair frequencies
    pair_freqs = compute_pair_freqs(word_freqs, splits_copy)


    # TODO: find the most frequent pair
    max_freq = 0
    most_frequent_pair = None
    for pair, freq in pair_freqs.items():
        if freq > max_freq:
            max_freq = freq
            most_frequent_pair = pair


    # TODO: update splits_copy by merging the most frequent pair
    splits_copy = merge_pair(most_frequent_pair[0], most_frequent_pair[1], splits_copy)

    # TODO: save merge rule
    merges_records[most_frequent_pair] = most_frequent_pair[0] + most_frequent_pair[1]
    vocab.append(most_frequent_pair[0] + most_frequent_pair[1])


    # TODO: append merged pair to vocab


CPU times: user 11min 59s, sys: 3.17 s, total: 12min 2s
Wall time: 12min 13s


Now, our BEP tokenizer is ready to use. To kokenize a string, simply pre-tokenize it and apply all merge rules in `merges_records`.

It's very important to know that the tokenization result of BPE tokenizer depends on the merging orders. The BPE algorithm learns the merge rules in a particular order based on the frequencies of subtokens. This ordering is used greedly during the encoding process for new text. That is why we want to iterate over `merges_records` in it's original order and apply merge rules one by one.

- **Problem 2.3.5-2**:
    - **Overview**: Implment a function `BPE_tokenize()` that takes in raw text and performs BPE tokenization to return a list of tokens.
    - **Input**: the text to be tokenized; `merges_records`
    - **Output**: a list of strings
    - **Requirements**:
        - Pre-tokenize the lowercased input text into a list of tokens, **think about which tokenizer to use**
        - Split each token in the list to a list of single characters
        - Iterate over `merges_records` to apply all merge rules to the splits
        - Return updated splits as a list of tokens
    - **Example**:

    ```
    >>> text = 'penpineapple-applepen!'
    >>> merges_records = {('i', 'n'): 'in', ('e', 'n'): 'en', ('l', 'e'): 'le',('a', 'p'): 'ap',\
    ('in', 'e'): 'ine', ('p', 'en'): 'pen', ('p', 'le'): 'ple', ('ap', 'ple'): 'apple', ('p', 'ine'): 'pine}
    >>> BPE_tokenize(text, merges_records)
    ['pen', 'pine', 'apple', '-', 'apple', 'pen', '!']
    ```



In [62]:
###### Part 2: Apply trained tokenizer ######
def BPE_tokenize(text, merges_records):
    """
    Use the merges_records to tokenize the input text

    @param text: the input text to be tokenized
    @return: a list of tokens
    """
    # TODO: pre-tokenize the lowercased input text, think about which tokenizer to use
    tokenized = nltk_tokenize_content(text.lower())

    # TODO: split each token in the list to a list of characters
    splits = [list(token) for token in tokenized]

    # TODO: iterate over merges_records and apply all merge rules
    for (a, b), ab in merges_records.items():
        splits = [merge_pair(a, b, {i: split})[i] for i, split in enumerate(splits)]
    return sum(splits, [])

In [63]:
### DO NOT CHANGE ###
tests = [
    "Antidisestablishmentarianism",
    "Conversationalization" ,
    "internationalization" ,
    "pronominalization" ,
    "personalization",
    "Most have seen this as Trump partisanizing the issue, much as he has with masks.",
    "partisanizing"
]

test_results = []
for test in tests:
    tmp_result = BPE_tokenize(test, merges_records)
    test_results.append(tmp_result)
    print(tmp_result)
### DO NOT CHANGE ###

# PennGrader - DO NOT CHANGE
grader.grade(test_case_id = 'test_q235_BPE_tokenize_function', answer = test_results)

['ant', 'id', 'is', 'establish', 'ment', 'arian', 'ism']
['con', 'vers', 'ation', 'al', 'ization']
['international', 'ization']
['pron', 'om', 'inal', 'ization']
['personal', 'ization']
['seen', 'trump', 'part', 'is', 'an', 'iz', 'ing', 'issue', 'much', 'mas', 'ks']
['part', 'is', 'an', 'iz', 'ing']
Correct! You earned 15/15 points. You are a star!

Your submission has been successfully recorded in the gradebook.


# End & Submission
### Congratulations on finishing your first homework! Here are the deliverables you need to submit to GradeScope
- This notebook and py file: rename to `homework1.ipynb` and `homework1.py`. You can download the notebook and py file by going to the top-left corner of this webpage, `File -> Download -> Download .ipynb/.py`